In [4]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [5]:
transac = pd.read_csv('../data/transac.csv', sep=';')
transac['date'] = pd.to_datetime(transac['date'])
transac['prod_price'] = transac['prod_price'].str.replace(',', '.').astype(float)
ref = pd.read_csv('../data/referentiel.csv', sep=';')

In [6]:
cnx = sqlite3.connect("../data/mdm.db")
transac.to_sql('TRANSACTIONS', con=cnx, index=False, if_exists='replace')
ref.to_sql('PRODUCT_NOMENCLATURE', con=cnx, index=False, if_exists='replace')

In [7]:
def run_query(query, cnx):
    cur = cnx.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    field_name = [field[0] for field in cur.description]
    data = pd.DataFrame(rows, columns=field_name)
    return data

In [8]:
query1 = """
select date,
       sum(prod_price * prod_qty) as CA
from TRANSACTIONS
where date between '2019-01-01' and '2019-12-31'
group by date
order by date
"""

In [9]:
run_query(query1, cnx)

,date,CA
0,2019-01-01 00:00:00,524.24
1,2019-01-02 00:00:00,524.24
2,2019-01-03 00:00:00,524.24


In [10]:
query2 = """
select client_id,
       sum(case when prd.product_type = 'MEUBLE' then prod_price * prod_qty end) as CA_MEUBLE_2019,
       sum(case when prd.product_type = 'DECO' then prod_price * prod_qty end) as CA_DECO_2019
from TRANSACTIONS trx left join PRODUCT_NOMENCLATURE prd on trx.prod_id = prd.product_id
where date between '2019-01-01' and '2019-12-31'
group by client_id
order by client_id
"""

In [11]:
run_query(query2, cnx)

,client_id,CA_MEUBLE_2019,CA_DECO_2019
0,845,1200.0,180.00
1,999,150.0,42.72
